# Bayesian preprocessing? (Rename this as needed)

Testing a renewal model with a Bayesian inference of parameters for preprocessing and estimating R(t).

In [154]:
import glob
import importlib
import os
from collections import defaultdict
from pathlib import Path
import re
import sys 

import epiweeks
import numpy as np
import pandas as pd
import plotly 
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots
import scipy
from plotly.subplots import make_subplots

from rtrend_forecast.visualization import plotly_fill_between

sys.path.append("../../../Rtrend_project/")
if os.path.basename(os.getcwd()) != "programs":
    os.chdir("..")  # Move to project root
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/pventura/NonCloudStorage/Flu_forecast/programs


In [158]:
# Example definition of parameters
# ================================
POP_RATE_REF = int(1E5)  # Reference constant for calculating rates. Eg: hospitalizations *per 100k people* 
WEEKLEN = 7  # Constant: number of days in a week

initial_model_params = dict(
    # Model params
    pop=1E6,  # State population
    c_pop_init=45,  # c(0) = c_pop_init * pop / 1E5. Initial rate of daily infection incidence
    t_init=pd.Timestamp("2024-01-01"),  # First day of *model* simulation
    t_present=pd.Timestamp("2024-01-01") + pd.Timedelta("7w"),
    roi_len=56,  # Number of days to simulate the *model* for.
    
    # Reproduction number model
    r_bias=0.00,
    drift_pop_coef=0.1E-5,  # drift = coef * 1E5 / pop * c(t)
    r_init=1.15,
    
    # Generation time
    tg_gamma_shape=10.,
    tg_gamma_rate=3.333333,
    tg_max=15,
    
    # Infection-to-hospitalization assigned report 
    hosp_delay_gamma_shape=25.,
    hosp_delay_gamma_rate=5.0,
    hosp_delay_max=15,
    hosp_fraction=0.01,
    likelihood_overdisp=0.0002,
)

mcmc_param_names = [
    "r_bias", 
    "r_init", 
    "c_pop_init", 
    "likelihood_overdisp"
]
mcmc_param_sigmas = dict(
    r_bias=0.0005,
    r_init=0.02,
    c_pop_init=0.5,
    likelihood_overdisp=0.01,
)
mcmc_param_bounds = dict(
    r_bias=(-np.inf, np.inf),
    r_init=(0, np.inf),
    c_pop_init=(0.01, POP_RATE_REF),
    likelihood_overdisp=(0.000000001, np.inf),
)

# ==============

class ModelParamsMain:
    c_pop_init: float
    pop: int
    r_bias: float
    drift_pop_coef: float
    r_init: float
    t_init: pd.Timestamp  # Date of the first simulated day. Start of the preprocessing ROI.
    roi_len: int
    tg_gamma_shape: float
    tg_gamma_rate: float
    tg_max: float
    hosp_delay_gamma_shape: float
    hosp_delay_gamma_rate: float
    hosp_delay_max: int
    hosp_fraction: float
    likelihood_overdisp: float

def run_model_once(ct_burnin_sr=None, **kwargs):
    p = ModelParamsMain()
    
    # Require all parameters created as annotations (type hints)
    for name in ModelParamsMain.__annotations__:
        setattr(p, name, kwargs[name])
        
    # Build the generation time distribution
    # ==================
    # CONVENTION: tg_pmf(s) = probab. of tg being of `s` days = CDF(s+1) - CDF(s)
    s_array = np.arange(1, p.tg_max)  # As of in Rtrend library, tg_max itself is also excluded.
    tg_gamma = scipy.stats.gamma(a=p.tg_gamma_shape, scale=1. / p.tg_gamma_rate)  # The continuous-time gamma that creates the TG PMF
    tg_pmf_array = np.diff(tg_gamma.cdf(s_array), append=tg_gamma.cdf(p.tg_max))  # This excludes the 0-1 day period
    tg_pmf_sr = pd.Series(tg_pmf_array, index=s_array) / tg_pmf_array.sum()
    
    
    # Renewal + Reproduction number model
    # =================
    
    # Burn-in period, just simulate a...
    # ----------------------------
    if ct_burnin_sr is None:
        # -()- constant incidence
        ct_burnin_index = pd.date_range(start=p.t_init - pd.Timedelta(p.tg_max, "D"), end=p.t_init - pd.Timedelta("1d"), freq="D")  # Stops one day before simulation period
        # c_init = int(round(p.c_pop_init * p.pop / POP_RATE_REF))
        c_init = float(p.c_pop_init * p.pop / POP_RATE_REF)
        ct_burnin_sr = pd.Series(c_init, index=ct_burnin_index)
        
        # -()- constant reproduction number
        # pass
    
    # Main ROI: Simulate a renewal model AND R(t) with drift, from t0 to t0 + roi_len
    # --------------------------
    ct_simulate_index = pd.date_range(start=p.t_init, end=p.t_init + pd.Timedelta(p.roi_len, "D"), freq="D")
    ct_simulate_sr = pd.Series(np.nan, index=ct_simulate_index)
    ct_sr = pd.concat([ct_burnin_sr, ct_simulate_sr])  # Incidence at all periods
    ct_current = ct_burnin_sr.iloc[-1]
    
    rt_sr = pd.Series(np.nan, index=ct_simulate_index)
    rt_current = p.r_init
    
    drift_coef = p.drift_pop_coef * POP_RATE_REF / p.pop
    
    for i_t, t in enumerate(ct_simulate_index):
        
        # Reproduction number
        rt_next = rt_current * np.exp(p.r_bias - drift_coef * ct_current)
        
        # Total infectiousness (convolution of past incidence with generation time)
        tot_infec = 0.
        for i_s, s in enumerate(tg_pmf_sr.index):
            tot_infec += ct_sr.loc[t - pd.Timedelta(s, "D")] * tg_pmf_sr[s]
        
        # Incidence of the next step
        ct_next = tot_infec * rt_next
        ct_sr.loc[t] = ct_next
        rt_sr.loc[t] = rt_next
        
        rt_current = rt_next
        ct_current = ct_next
            
    return ct_sr, rt_sr


def calculate_loglikelihood(ct_sr: pd.Series, rt_sr: pd.Series, data_sr: pd.Series, weekday="Saturday", **kwargs):
    p = ModelParamsMain()
    
    # Require all parameters created as annotations (type hints)
    for name in ModelParamsMain.__annotations__:
        setattr(p, name, kwargs[name])
            
    # =======
        
    # Check whether dates in the data are Saturdays
    if not (data_sr.index.day_name() == weekday).all():
        raise ValueError(f"Data should be `weekday`= {weekday}")

    # Check whether all dates in the data are in the model series
    if not data_sr.index.isin(ct_sr.index).all():
        raise ValueError(f"All dates in data should be contained in the model series")

    # =========
    
    # Build the infection-to-hospitalization delay distribution
    # ==================
    # CONVENTION: hosp_pmf(s) = probab. of an infection at day 0 being hospitalized at day s = CDF(s+1) - CDF(s)
    s_array = np.arange(1, p.hosp_delay_max)  # As of in Rtrend library, tg_max itself is also excluded.
    hosp_gamma = scipy.stats.gamma(a=p.hosp_delay_gamma_shape, scale=1. / p.hosp_delay_gamma_rate)  # The continuous-time gamma that creates the TG PMF
    hosp_pmf_array = np.diff(hosp_gamma.cdf(s_array), append=hosp_gamma.cdf(p.hosp_delay_max))  # For simplicity, it also excludes the 0-1 day period
    hosp_pmf_sr = pd.Series(hosp_pmf_array, index=s_array) / hosp_pmf_array.sum()
    
    # --- Process and Aggregate model outputs
    # ==================
    # fit_window_daily_index = pd.DatetimeIndex(
    #     [day for week_end_date in data_sr.index for day in pd.date_range(week_end_date - pd.Timedelta(days=6), week_end_date, freq="D")]
    # )
    
    # --- Calculate the weekly series of _expectancy_ (average) of hospitalizations from the model
    # For each week of data...
    mu_sr = pd.Series(np.nan, index=data_sr.index, name="expectancy_hosp")  # Expectancy of weekly hospitalizations, according to the model
    for i_w, w in enumerate(data_sr.index):
        w: pd.Timestamp
        mu_w = 0.  # Expectancy of weekly hospitalizations
        week_daily_index = pd.date_range(end=w, freq="D", periods=WEEKLEN)
        
        # For each day in the week...
        for i_t, t in enumerate(week_daily_index):
            # For each past lag `s` in days...
            for i_s, s in enumerate(hosp_pmf_sr.index):
                # c(t - s) * f_h * eta(s)
                mu_w += p.hosp_fraction * ct_sr.loc[t - pd.Timedelta(s, "D")] * hosp_pmf_sr.loc[s]
                
        mu_sr.loc[w] = mu_w
        
    # --- Calculate the overall loglikelihood
    n = 1 / p.likelihood_overdisp
    p_sr = n / (n + mu_sr)  # 
    ll_sr = scipy.stats.nbinom.logpmf(k=data_sr, n=n, p=p_sr)
    total_ll = ll_sr.sum()

    return mu_sr, total_ll
    

In [75]:
# Synthesize some data
# =========    
t_present = initial_model_params["t_present"]
n_weeks_fit_roi = 6  # Number of weeks of latest data to for the model for
# epiweeks.Week(t_present.year, t_present.week)
# Get the reference dates
present_week_end_date = epiweeks.Week.fromdate(t_present).enddate()  # Last epiweek of data

# Let's make some data up to present week
_rng = np.random.default_rng(0)
data_t_index = pd.date_range(end=present_week_end_date, freq="W-SAT", periods=n_weeks_fit_roi, name="date")
data_values = np.round(50 * np.exp(0.35 * np.arange(n_weeks_fit_roi))).astype(int)
data_values += _rng.normal(scale=90, size=data_values.shape).round().astype(int)

# data_values = np.round(100 * np.exp(0.35 * np.arange(n_weeks_fit_roi))).astype(int)

data_roi_sr = pd.Series(data_values, index=data_t_index, name="value")

(data_roi_sr.index.day_name() == "Saturday").all()
   # Check whether dates are Saturdays

# ========
# Run test and visualize
_ct_sr, _rt_sr = run_model_once(**initial_model_params)
_mu_sr, _total_ll = calculate_loglikelihood(_ct_sr, _rt_sr, data_roi_sr, **initial_model_params)

# Let's recreate the Negative Binomial model of hospital reports
_nb_n = 1. / initial_model_params["likelihood_overdisp"]
_nb_p_list = [_nb_n / (_nb_n + mu) for mu in _mu_sr]
nb_dist_list = [scipy.stats.nbinom(n=_nb_n, p=_p) for _p in _nb_p_list]  #  A list of frozen distributions, one for each week
# And now we can extract some stats from it. It represents uncertainty in the hospitalization and reporting process.
nb_avg_array = np.fromiter((dist.mean() for dist in nb_dist_list), dtype=float)
nb_median_array = np.fromiter((dist.median() for dist in nb_dist_list), dtype=float)

# display(_mu_sr)  # Model expectancy
# display(data_roi_sr)  # Empirical data
print(f"Total likelihood = {_total_ll}")


# ----------
fig = plotly.subplots.make_subplots(rows=2, cols=1, row_heights=[800, 600], shared_xaxes=True)

# fig.add_scatter(x=_ct_sr.index, y=_ct_sr.values, row=1, col=1, name="Model infection incidence")
plotly_fill_between(fig=fig, x=_mu_sr.index, 
                    y1=np.fromiter((dist.ppf(0.025) for dist in nb_dist_list), dtype=float),
                    y2=np.fromiter((dist.ppf(0.975) for dist in nb_dist_list), dtype=float),
                    name="NB 95% IQR", line_width=0., fillcolor="olivedrab",
                    )
fig.add_scatter(x=_mu_sr.index, y=_mu_sr.values, row=1, col=1, name="Model avg. hospitalization incidence")
fig.add_scatter(x=data_roi_sr.index, y=data_roi_sr.values, row=1, col=1, name="Empirical data")
fig.add_scatter(x=_rt_sr.index, y=_rt_sr.values, row=2, col=1, name="R(t)")
fig.add_hline(y=1, row=2, col=1)

fig.show()

Total likelihood = -108.25210654895426


## Run an MCMC loop

In [82]:
# DEFINE AND RUN THE BASIC MCMC PROCEDURE
# ===================================

def run_mcmc_loop(
        initial_params:dict,
        data_sr:pd.Series,
        n_steps_burn=100, n_steps_store=1000,
        seed=10
):
    
    # Initialize structures
    rng = np.random.default_rng(seed)
    curr_params = initial_params.copy()  # Deepcopy needed?
    curr_likelihood = -np.inf
    cand_params = curr_params.copy()
    
    def run_mcmc_step(_curr_likelihood):
        # Sample candidate parameters
        feasible = True
        for name in mcmc_param_names:
            # Sample (Gaussian step)
            cand_param = curr_params[name] + rng.normal(scale=mcmc_param_sigmas[name])
            # Check candidate feasibility (bounds)
            if not (mcmc_param_bounds[name][0] <= cand_param <= mcmc_param_bounds[name][1]):
                feasible = False
            cand_params[name] = cand_param

        # Run model and calc likelihood with candidate
        if not feasible:
            _cand_likelihood = -np.inf
        else:
            ct_sr, rt_sr = run_model_once(**cand_params)
            mu_sr, _cand_likelihood = calculate_loglikelihood(ct_sr, rt_sr, data_sr, **cand_params)
            
        # Return: candidate likelihood, whether it should be accepted
        # `cand_params` is modified in place
        return _cand_likelihood, rng.random() < np.exp(_cand_likelihood - _curr_likelihood)
        
        # # Choose between candidate and current, then update 
        # if rng.random() < np.exp(_cand_likelihood - _curr_likelihood):
        #     # Accepted: update params and return new likelihood
        #     for name in mcmc_param_names:
        #         curr_params[name] = cand_params[name]
        #     return _cand_likelihood, True
        # else:
        #     # Rejected: return current likelihood
        #     return _curr_likelihood, False
        
        
    # Burn-in MCMC loop
    for i_step in range(0, n_steps_burn):
        cand_likelihood, accepted = run_mcmc_step(curr_likelihood)
        
        # Update
        if accepted:
            for name in mcmc_param_names:
                curr_params[name] = cand_params[name]
            curr_likelihood = cand_likelihood
    
    # Main MCMC loop
    likelihood_log = {f"{name}_loglikelihood": list() for name in ["curr", "cand"]}
    params_log = defaultdict(lambda: list())  # {name: list() for name in mcmc_param_names}
    accepted_log = list()
    for i_step in range(0, n_steps_store):
        cand_likelihood, accepted = run_mcmc_step(curr_likelihood)

        # Store in the logs
        likelihood_log["curr_loglikelihood"].append(curr_likelihood)  # Stores the _candidate_ if accepted, _current_ if not
        likelihood_log["cand_loglikelihood"].append(cand_likelihood)  # Stores the _candidate_ if accepted, _current_ if not
        for name in mcmc_param_names:
            params_log[name].append(curr_params[name])
        for name in mcmc_param_names:
            params_log[f"{name}_cand"].append(cand_params[name])
        accepted_log.append(accepted)
        
        # Update
        if accepted:
            for name in mcmc_param_names:
                curr_params[name] = cand_params[name]
            curr_likelihood = cand_likelihood
            
        if n_steps_store > 1000 and i_step % 500 == 0:
            print(f"Step {i_step + 1} of {n_steps_store} in the store period")
    
    # Assemble the results as a data frame
    out_df = pd.DataFrame(dict(**params_log, **likelihood_log, accepted=accepted_log))
    # out_df["loglikelihood"] = likelihood_log

    return out_df
    
# =============
# Run a quick test
mcmc_results_df = run_mcmc_loop(initial_model_params, data_roi_sr, n_steps_burn=0, n_steps_store=200)

mcmc_results_df

r_bias    r_init  c_pop_init  likelihood_overdisp  r_bias_cand  \
0    0.000000  1.150000   45.000000             0.000200    -0.000552   
1   -0.000552  1.135500   44.609097             0.002870    -0.000488   
2   -0.000552  1.135500   44.609097             0.002870    -0.000929   
3   -0.000552  1.135500   44.609097             0.002870    -0.001119   
4   -0.000552  1.135500   44.609097             0.002870    -0.001044   
..        ...       ...         ...                  ...          ...   
195  0.001765  1.093161   43.452660             0.058296     0.002251   
196  0.001765  1.093161   43.452660             0.058296     0.001340   
197  0.001765  1.093161   43.452660             0.058296     0.002396   
198  0.001765  1.093161   43.452660             0.058296     0.002023   
199  0.001765  1.093161   43.452660             0.058296     0.001552   

     r_init_cand  c_pop_init_cand  likelihood_overdisp_cand  \
0       1.135500        44.609097                  0.002870   
1       1.152360        45.038066                  0.007622   
2       1.119203        44.437170                  0.002356   
3       1.141614        43.683255                  0.001099   
4       1.113240        44.228784                  0.009350   
..           ...              ...                       ...   
195     1.106163        43.061984                  0.068713   
196     1.076039        43.265337                  0.062551   
197     1.098589        43.905051                  0.053715   
198     1.135938        43.904172                  0.073020   
199     1.095940        43.829066                  0.060163   

     curr_loglikelihood  cand_loglikelihood  accepted  
0                  -inf          -37.101242      True  
1            -37.101242          -42.912927     False  
2            -37.101242          -77.425321     False  
3            -37.101242          -48.808845     False  
4            -37.101242          -70.825253     False  
..                  ...                 ...       ...  
195          -30.130807          -34.732974     False  
196          -30.130807          -32.242624     False  
197          -30.130807          -35.133904     False  
198          -30.130807          -43.394540     False  
199          -30.130807          -29.831114      True  

[200 rows x 11 columns]

In [86]:
# fig = px.line(mcmc_results_df, y=mcmc_results_df.columns)
# fig.show()

plot_variables = [col for col in mcmc_results_df.columns if (col not in ["accepted"] and "_cand" not in col)]
num_panels = len(plot_variables)

fig = plotly.subplots.make_subplots(rows=num_panels, cols=1, shared_xaxes=True)

for i_subplot, col_name in enumerate(plot_variables):
    fig.add_scatter(y=mcmc_results_df[col_name], name=col_name, row=i_subplot+1, col=1)
    fig.update_yaxes(title_text=col_name, row=i_subplot+1, col=1)

fig.update_layout(
    height=1000
)
fig.show()
# mcmc_results_df

In [88]:
# Let's first check the model with maximum likelihood
# ------------------------------
# Get maximum likelihood and its parameters
max_ll_id = mcmc_results_df["curr_loglikelihood"].idxmax()
row = mcmc_results_df.loc[max_ll_id]
max_ll_params = initial_model_params.copy()
max_ll_params.update(row)

# Run test and visualize
_ct_sr, _rt_sr = run_model_once(**max_ll_params)
_mu_sr, _total_ll = calculate_loglikelihood(_ct_sr, _rt_sr, data_roi_sr, **max_ll_params)

# Let's recreate the Negative Binomial model of hospital reports
_nb_n = 1. / initial_model_params["likelihood_overdisp"]
_nb_p_list = [_nb_n / (_nb_n + mu) for mu in _mu_sr]
nb_dist_list = [scipy.stats.nbinom(n=_nb_n, p=_p) for _p in _nb_p_list]  #  A list of frozen distributions, one for each week
# And now we can extract some stats from it. It represents uncertainty in the hospitalization and reporting process.
nb_avg_array = np.fromiter((dist.mean() for dist in nb_dist_list), dtype=float)
nb_median_array = np.fromiter((dist.median() for dist in nb_dist_list), dtype=float)

# display(_mu_sr)  # Model expectancy
# display(data_roi_sr)  # Empirical data
print(f"Total likelihood = {_total_ll}")

# ----------
from rtrend_forecast.visualization import plotly_fill_between
fig = plotly.subplots.make_subplots(rows=2, cols=1, row_heights=[800, 600], shared_xaxes=True)

# fig.add_scatter(x=_ct_sr.index, y=_ct_sr.values, row=1, col=1, name="Model infection incidence")
plotly_fill_between(fig=fig, x=_mu_sr.index, 
                    y1=np.fromiter((dist.ppf(0.025) for dist in nb_dist_list), dtype=float),
                    y2=np.fromiter((dist.ppf(0.975) for dist in nb_dist_list), dtype=float),
                    name="NB 95% IQR", line_width=0., fillcolor="olivedrab",
                    )
fig.add_scatter(x=_mu_sr.index, y=_mu_sr.values, row=1, col=1, name="Model avg. hospitalization incidence")
fig.add_scatter(x=data_roi_sr.index, y=data_roi_sr.values, row=1, col=1, name="Empirical data")
fig.add_scatter(x=_rt_sr.index, y=_rt_sr.values, row=2, col=1, name="R(t)")
fig.add_hline(y=1, row=2, col=1)

Total likelihood = -29.01388878952093


In [94]:
# Let's run the model with the MCMC'd parameters
# ========================================
i_step_start = 20 # First MCMC step to take
model_params = initial_model_params.copy()

ct_sr_list = list()
rt_sr_list = list()
for i_step, row in mcmc_results_df.iloc[i_step_start:].iterrows():
    model_params.update(row)
    
    _ct_sr, _rt_sr = run_model_once(**model_params)
    ct_sr_list.append(_ct_sr)
    rt_sr_list.append(_rt_sr)
    
# NOTICE!!! From c(t), we'd still have to apply the reporting delays and fractions to obtain hospitalizations!
# So the plot below is not comparing things fairly
    
ct_ensemble_df = pd.concat(ct_sr_list, axis=1)  # Signature: df.loc[date, i_sample] = c(t)
rt_ensemble_df = pd.concat(rt_sr_list, axis=1)

ct_ensemble_df

0            1            2            3            4     \
2023-12-17   450.957908   455.637860   455.637860   452.078537   443.258593   
2023-12-18   450.957908   455.637860   455.637860   452.078537   443.258593   
2023-12-19   450.957908   455.637860   455.637860   452.078537   443.258593   
2023-12-20   450.957908   455.637860   455.637860   452.078537   443.258593   
2023-12-21   450.957908   455.637860   455.637860   452.078537   443.258593   
...                 ...          ...          ...          ...          ...   
2024-02-22  4602.643886  4984.195907  4984.195907  5039.439017  6092.856277   
2024-02-23  4749.003121  5177.406224  5177.406224  5193.069499  6358.250576   
2024-02-24  4897.505553  5376.323379  5376.323379  5348.039099  6632.882249   
2024-02-25  5048.033449  5580.986276  5580.986276  5504.157305  6916.874680   
2024-02-26  5200.459126  5791.422926  5791.422926  5661.222665  7210.333661   

                   5            6            7            8            9     \
2023-12-17   443.258593   443.258593   446.914827   448.100553   448.100553   
2023-12-18   443.258593   443.258593   446.914827   448.100553   448.100553   
2023-12-19   443.258593   443.258593   446.914827   448.100553   448.100553   
2023-12-20   443.258593   443.258593   446.914827   448.100553   448.100553   
2023-12-21   443.258593   443.258593   446.914827   448.100553   448.100553   
...                 ...          ...          ...          ...          ...   
2024-02-22  6092.856277  6092.856277  4069.846877  3933.773899  3933.773899   
2024-02-23  6358.250576  6358.250576  4190.272709  4073.002857  4073.002857   
2024-02-24  6632.882249  6632.882249  4312.103560  4215.999747  4215.999747   
2024-02-25  6916.874680  6916.874680  4435.234992  4362.791089  4362.791089   
2024-02-26  7210.333661  7210.333661  4559.555535  4513.397776  4513.397776   

            ...         1970         1971         1972         1973  \
2023-12-17  ...   692.023202   692.023202   688.715033   687.815496   
2023-12-18  ...   692.023202   692.023202   688.715033   687.815496   
2023-12-19  ...   692.023202   692.023202   688.715033   687.815496   
2023-12-20  ...   692.023202   692.023202   688.715033   687.815496   
2023-12-21  ...   692.023202   692.023202   688.715033   687.815496   
...         ...          ...          ...          ...          ...   
2024-02-22  ...  2606.194167  2606.194167  2185.123171  3540.573393   
2024-02-23  ...  2524.679154  2524.679154  2115.302168  3474.143106   
2024-02-24  ...  2440.813468  2440.813468  2043.789351  3402.913284   
2024-02-25  ...  2355.017363  2355.017363  1970.919780  3327.246464   
2024-02-26  ...  2267.706986  2267.706986  1897.023674  3247.520315   

                   1974         1975         1976         1977         1978  \
2023-12-17   696.672605   702.451282   702.451282   702.451282   697.362155   
2023-12-18   696.672605   702.451282   702.451282   702.451282   697.362155   
2023-12-19   696.672605   702.451282   702.451282   702.451282   697.362155   
2023-12-20   696.672605   702.451282   702.451282   702.451282   697.362155   
2023-12-21   696.672605   702.451282   702.451282   702.451282   697.362155   
...                 ...          ...          ...          ...          ...   
2024-02-22  3064.781836  2442.529363  2442.529363  2442.529363  2349.354589   
2024-02-23  2995.733994  2389.437141  2389.437141  2389.437141  2285.348745   
2024-02-24  2922.973091  2333.724492  2333.724492  2333.724492  2219.111346   
2024-02-25  2846.860144  2275.637894  2275.637894  2275.637894  2150.949015   
2024-02-26  2767.764274  2215.428837  2215.428837  2215.428837  2081.168714   

                   1979  
2023-12-17   697.362155  
2023-12-18   697.362155  
2023-12-19   697.362155  
2023-12-20   697.362155  
2023-12-21   697.362155  
...                 ...  
2024-02-22  2349.354589  
2024-02-23  2285.348745  
2024-02-24  2219.111346  
2024-02-25  2150.949015  
2024-02-26  2081.168714  

[72 ro

In [95]:
fig = plotly.subplots.make_subplots(rows=2, cols=1)

# Calculate some stats
ct_median = ct_ensemble_df.median(axis=1)
ct_low_q = ct_ensemble_df.quantile(0.025, axis=1)
ct_high_q = ct_ensemble_df.quantile(0.975, axis=1)

rt_median = rt_ensemble_df.median(axis=1)
rt_low_q = rt_ensemble_df.quantile(0.025, axis=1)
rt_high_q = rt_ensemble_df.quantile(0.975, axis=1)

fig.add_scatter(x=data_roi_sr.index, y=data_roi_sr.values, name="Empirical data")
# fig.add_scatter(x=ct_median.index, y=ct_median, name="Model median")
fig.add_scatter(x=ct_median.index, y=ct_median * initial_model_params["hosp_fraction"], name="Model median * hosp_frac")

fig.add_scatter(x=rt_median.index, y=rt_median, name="R(t) median", row=2, col=1)
plotly_fill_between(fig, x=rt_median.index, y1=rt_low_q, y2=rt_high_q, row=2, col=1, name="RT 95% IQR", line_width=0)
# fig.add_scatter(x=rt_median.index, y=rt_median, name="R(t) median")

fig.show()

# Test the model with real Flu hospitalization data

In [21]:
# Load FluSight truth and locations dataset
# ==================
import utils.flu_misc_utils as misc_utils
importlib.reload(misc_utils)

truth_big_df, locations_df = misc_utils.load_entire_flu_dataset()

truth_big_df

2023-11-11: Saturday
2023-11-04: Saturday
2024-02-24: Saturday
2024-04-20: Saturday
2023-12-09: Saturday
2024-03-02: Saturday
2024-03-16: Saturday
2024-01-13: Saturday
2023-12-23: Saturday
2024-01-06: Saturday
2024-04-27: Saturday
2023-12-30: Saturday
2023-10-21: Saturday
2024-03-09: Saturday
2023-09-30: Saturday
2024-03-23: Saturday
2023-10-07: Saturday
2023-12-02: Saturday
2023-12-16: Saturday
2024-01-27: Saturday
2023-09-23: Saturday
2024-04-13: Saturday
2024-04-06: Saturday
2024-01-20: Saturday
2023-10-14: Saturday
2023-10-28: Saturday
2024-03-30: Saturday
2024-02-03: Saturday
2024-02-17: Saturday
2023-11-18: Saturday
2023-11-25: Saturday
2024-02-10: Saturday
Loading all truth data:


location  value  weekly_rate  horizon
location_name report_date date                                            
Alaska        2023-09-23  2023-09-23       02      5     0.702814   0 days
Alabama       2023-09-23  2023-09-23       01     11     0.217229   0 days
Arkansas      2023-09-23  2023-09-23       05      6     0.197247   0 days
Arizona       2023-09-23  2023-09-23       04      9     0.122599   0 days
California    2023-09-23  2023-09-23       06     51     0.131151   0 days
...                                       ...    ...          ...      ...
Vermont       2024-04-27  2022-02-12       50      0     0.000000 805 days
Washington    2024-04-27  2022-02-12       53      8     0.103415 805 days
Wisconsin     2024-04-27  2022-02-12       55     29     0.492275 805 days
West Virginia 2024-04-27  2022-02-12       54      5     0.281694 805 days
Wyoming       2024-04-27  2022-02-12       56      2     0.345672 805 days

[170448 rows x 4 columns]

In [205]:
# Test the fit with one location and reference date
# ============
location_name = "Florida"
report_date = pd.Timestamp(epiweeks.Week.fromstring("2024-01-02").enddate())
_model_run_days = 49  # This will set "roi_len". Must be greater than fit window in weeks.
_data_fit_window_weeks = 7 - 1  # This will just be used to crop the data. (SUBTRACT ONE because DataFrame.loc[] is INCLUSIVE).


# Change sigmas for this run (affects all runs!)
mcmc_param_sigmas = dict(
    r_bias=0.0005,
    r_init=0.02,
    c_pop_init=0.75,
    likelihood_overdisp=0.01,
)

# ---------

# --- Copy and modify the initial parameters
fs_initial_params = initial_model_params.copy()
fs_initial_params.update(dict(
    pop=locations_df.set_index("location_name").loc[location_name, "population"],
    # t_present=report_date,  # Last epiweek of data for synthetic data. Let's not use it here.
    t_init=report_date - pd.Timedelta(_model_run_days, "d"),
    roi_len=_model_run_days,  # Nr of days to simulate the model for.
    
    c_pop_init=5.5,
    
    # Reproduction number    
    drift_pop_coef=8.0E-5,  # drift = coef * 1E5 / pop * c(t)
))
# Calculate the first day to run the model for based on the reference date and the specified ROI length
model_extra_days_before = 15  # Run the model for this many days before the fit window
# fs_initial_params["t_init"] = report_date - pd.Timedelta(_model_run_days)

# --- Cross-section truth data
print(f"Report (reference) date = {report_date.date().isoformat()}")
_xs_df = truth_big_df.xs((location_name, report_date), level=["location_name", "report_date"]).sort_index()
_raw_incid_sr = _xs_df["value"]  # Full series
_fit_window_start = report_date - pd.Timedelta(_data_fit_window_weeks, "w")
_roi_incid_sr = _raw_incid_sr.loc[_fit_window_start:report_date]  # Model Fit Window (ROI)

# display(_roi_incid_sr.loc[pd.Timestamp('2023-12-23 00:00:00')])
# display(_roi_incid_sr)


# =============

# # --- Test the model once with initial parameters
# ct_sr, rt_sr = run_model_once(**fs_initial_params)
# display(fs_initial_params["hosp_fraction"] * ct_sr * WEEKLEN)
# display(_roi_incid_sr)

# --- Run MCMC
fs_mcmc_results_df = run_mcmc_loop(
    initial_params=fs_initial_params,
    data_sr=_roi_incid_sr,
    n_steps_burn=50,
    n_steps_store=5000,
    # n_steps_store=100,
)

fs_mcmc_results_df

Report (reference) date = 2024-01-06
Step 1 of 5000 in the store period
Step 501 of 5000 in the store period
Step 1001 of 5000 in the store period
Step 1501 of 5000 in the store period
Step 2001 of 5000 in the store period
Step 2501 of 5000 in the store period
Step 3001 of 5000 in the store period
Step 3501 of 5000 in the store period
Step 4001 of 5000 in the store period
Step 4501 of 5000 in the store period


r_bias    r_init  c_pop_init  likelihood_overdisp  r_bias_cand  \
0     0.000954  1.170941   12.359975             0.064918     0.001556   
1     0.001556  1.168724   12.758816             0.078566     0.001326   
2     0.001556  1.168724   12.758816             0.078566     0.001872   
3     0.001556  1.168724   12.758816             0.078566     0.001936   
4     0.001936  1.166607   13.383014             0.071162     0.001625   
...        ...       ...         ...                  ...          ...   
4995  0.006738  0.975093   46.830408             0.003014     0.006664   
4996  0.006664  0.992562   46.745574             0.012337     0.007635   
4997  0.006664  0.992562   46.745574             0.012337     0.006362   
4998  0.006664  0.992562   46.745574             0.012337     0.006430   
4999  0.006664  0.992562   46.745574             0.012337     0.008051   

      r_init_cand  c_pop_init_cand  likelihood_overdisp_cand  \
0        1.168724        12.758816                  0.078566   
1        1.128965        12.952929                  0.074434   
2        1.177823        11.529309                  0.080104   
3        1.166607        13.383014                  0.071162   
4        1.161334        13.140581                  0.082111   
...           ...              ...                       ...   
4995     0.992562        46.745574                  0.012337   
4996     0.989126        45.756187                  0.029135   
4997     1.022351        47.105309                  0.004172   
4998     1.025581        47.399848                  0.003825   
4999     0.996639        46.314478                  0.006534   

      curr_loglikelihood  cand_loglikelihood  accepted  
0             -67.664583          -63.311913      True  
1             -63.311913          -69.389243     False  
2             -63.311913          -67.442646     False  
3             -63.311913          -63.158603      True  
4             -63.158603          -61.920617      True  
...                  ...                 ...       ...  
4995          -39.411384          -40.800747      True  
4996          -40.800747          -45.536345     False  
4997          -40.800747          -69.052040     False  
4998          -40.800747          -80.943876     False  
4999          -40.800747          -71.956867     False  

[5000 rows x 11 columns]

In [206]:
plot_variables = [col for col in fs_mcmc_results_df.columns if (col not in ["accepted"] and "cand" not in col)]
num_panels = len(plot_variables)

fig = plotly.subplots.make_subplots(rows=num_panels, cols=1, shared_xaxes=True)

for i_subplot, col_name in enumerate(plot_variables):
    fig.add_scatter(y=fs_mcmc_results_df[col_name], name=col_name, row=i_subplot+1, col=1)
    fig.update_yaxes(title_text=col_name, row=i_subplot+1, col=1)

fig.update_layout(
    height=1000
)
fig.show()

In [207]:
# Maximum likelihood model(s)
# ------------------------------
# -()- Get maximum likelihood and its parameters
max_ll_id = fs_mcmc_results_df["curr_loglikelihood"].idxmax()
row = fs_mcmc_results_df.loc[max_ll_id]
max_ll_params = fs_initial_params.copy()
max_ll_params.update(row)


# --------
# Run test and visualize
_ct_sr, _rt_sr = run_model_once(**max_ll_params)
_mu_sr, _total_ll = calculate_loglikelihood(_ct_sr, _rt_sr, _roi_incid_sr, **max_ll_params)

# # Let's recreate the Negative Binomial model of hospital reports
# _nb_n = 1. / initial_model_params["likelihood_overdisp"]
# _nb_p_list = [_nb_n / (_nb_n + mu) for mu in _mu_sr]
# nb_dist_list = [scipy.stats.nbinom(n=_nb_n, p=_p) for _p in _nb_p_list]  #  A list of frozen distributions, one for each week
# # And now we can extract some stats from it. It represents uncertainty in the hospitalization and reporting process.
# nb_avg_array = np.fromiter((dist.mean() for dist in nb_dist_list), dtype=float)
# nb_median_array = np.fromiter((dist.median() for dist in nb_dist_list), dtype=float)
# 
# # display(_mu_sr)  # Model expectancy
# # display(data_roi_sr)  # Empirical data
# print(f"Total likelihood = {_total_ll}")

# ----------
from rtrend_forecast.visualization import plotly_fill_between
fig = plotly.subplots.make_subplots(rows=2, cols=1, row_heights=[800, 600], shared_xaxes=True)

# fig.add_scatter(x=_ct_sr.index, y=_ct_sr.values, row=1, col=1, name="Model infection incidence")
# plotly_fill_between(fig=fig, x=_mu_sr.index, 
#                     y1=np.fromiter((dist.ppf(0.025) for dist in nb_dist_list), dtype=float),
#                     y2=np.fromiter((dist.ppf(0.975) for dist in nb_dist_list), dtype=float),
#                     name="NB 95% IQR", line_width=0., fillcolor="olivedrab",
#                     )
fig.add_scatter(x=_mu_sr.index, y=_mu_sr.values, row=1, col=1, name="Model avg. hospitalization incidence")
fig.add_scatter(x=_roi_incid_sr.index, y=_roi_incid_sr.values, row=1, col=1, name="Empirical data")
fig.add_scatter(x=_rt_sr.index, y=_rt_sr.values, row=2, col=1, name="R(t)")
fig.add_hline(y=1, row=2, col=1)
fig.show()

In [208]:
# Select a subset of MCMC models to run again
# ====================

# -()- Get a random sample of models
_mcmc_rows = fs_mcmc_results_df.sample(100, replace=False, random_state=15)

# # -()- Get the best likelihood models
# _mcmc_rows = fs_mcmc_results_df.sort_values("curr_loglikelihood", ascending=False).iloc[:10]

display(_mcmc_rows)

# ----

rnd_sample_params = fs_initial_params.copy()
_ct_sr_list = list()
_mu_sr_list = list()
_rt_sr_list = list()
_keys_list = list()
for i, row in _mcmc_rows.iterrows():
    rnd_sample_params.update(row)
    
    # Run model 
    _ct_sr, _rt_sr = run_model_once(**rnd_sample_params)
    _mu_sr, _total_ll = calculate_loglikelihood(_ct_sr, _rt_sr, _roi_incid_sr, **rnd_sample_params)
    
    _ct_sr_list.append(_ct_sr)
    _mu_sr_list.append(_mu_sr)
    _rt_sr_list.append(_rt_sr)
    _keys_list.append(i)
    
_ct_runs_df = pd.concat(_ct_sr_list, axis=1, keys=_keys_list)
_mu_runs_df = pd.concat(_mu_sr_list, axis=1, keys=_keys_list)
_rt_runs_df = pd.concat(_rt_sr_list, axis=1, keys=_keys_list)

_mu_runs_df

    

r_bias    r_init  c_pop_init  likelihood_overdisp  r_bias_cand  \
2835  0.006155  0.996357   43.629527             0.023365     0.005846   
976   0.005234  1.024505   40.472122             0.001681     0.005467   
3343  0.005288  1.012657   44.202426             0.011453     0.005268   
4519  0.007034  0.975973   49.405509             0.002552     0.007046   
4185  0.007904  0.946684   56.934619             0.006716     0.008426   
...        ...       ...         ...                  ...          ...   
3026  0.005727  1.004245   41.654623             0.004394     0.005865   
275   0.001786  1.166763   31.030904             0.149515     0.001604   
2463  0.006003  1.002674   45.566074             0.003049     0.005205   
263   0.001609  1.110304   31.150504             0.163812     0.001624   
3664  0.007082  0.978027   63.810314             0.035432     0.006454   

      r_init_cand  c_pop_init_cand  likelihood_overdisp_cand  \
2835     1.001149        44.387606                  0.024979   
976      1.016855        40.515356                  0.002756   
3343     1.046591        43.714988                  0.004314   
4519     0.976122        48.232860                  0.018371   
4185     0.906797        57.467583                 -0.006420   
...           ...              ...                       ...   
3026     0.993890        40.599067                 -0.011992   
275      1.160194        31.156048                  0.153250   
2463     0.991655        45.973899                 -0.009700   
263      1.080041        33.062714                  0.165772   
3664     0.986199        63.573798                  0.033171   

      curr_loglikelihood  cand_loglikelihood  accepted  
2835          -41.630870          -41.936432      True  
976           -40.450343          -38.377646      True  
3343          -40.506330          -65.937174     False  
4519          -38.098412          -41.023913     False  
4185          -42.444980                -inf     False  
...                  ...                 ...       ...  
3026          -38.346592                -inf     False  
275           -50.870115          -50.270899      True  
2463          -38.503074                -inf     False  
263           -48.839884          -49.624507     False  
3664          -46.762805          -47.110487      True  

[100 rows x 11 columns]

2835         976          3343         4519         4185  \
date                                                                          
2023-11-25   691.574419   653.509185   707.640308   772.588274   872.690422   
2023-12-02   713.082883   717.565427   750.765067   759.661745   796.551965   
2023-12-09   772.116933   814.305898   820.593685   792.669403   779.345824   
2023-12-16   872.750483   942.884296   916.003853   876.955477   823.931976   
2023-12-23  1018.843353  1099.288505  1034.035386  1019.259114   936.387200   
2023-12-30  1209.725237  1270.237936  1166.394107  1225.748763  1129.922816   
2024-01-06  1432.613866  1430.533709  1297.572864  1492.530038  1417.693299   

                   2694         2238         536          60           2680  \
date                                                                          
2023-11-25   697.141333   673.143397   641.028846   384.435242   694.705299   
2023-12-02   691.149667   711.641842   676.251615   503.195172   698.032793   
2023-12-09   734.195907   787.141949   743.463889   667.452531   746.604377   
2023-12-16   834.055678   902.945598   845.051610   873.178964   847.224673   
2023-12-23  1002.254394  1061.162941   982.515744  1103.363105  1008.925079   
2023-12-30  1251.068482  1256.870958  1151.992496  1316.879401  1239.264493   
2024-01-06  1580.074808  1470.932398  1338.941627  1455.819589  1532.635283   

            ...         453          2741         2755         2184  \
date        ...                                                       
2023-11-25  ...   639.423186   707.848314   695.615283   651.372124   
2023-12-02  ...   734.697530   718.502801   688.053118   673.043908   
2023-12-09  ...   842.558256   776.964808   728.307997   739.798305   
2023-12-16  ...   948.802692   890.786210   823.680247   859.471992   
2023-12-23  ...  1038.476607  1069.527993   984.971414  1041.820381   
2023-12-30  ...  1095.626075  1319.414330  1223.932089  1292.342769   
2024-01-06  ...  1108.786367  1629.441545  1540.927077  1598.428256   

                   2253         3026         275          2463         263   \
date                                                                          
2023-11-25   672.112244   663.626082   550.054516   725.108338   531.143858   
2023-12-02   708.345817   695.688017   818.871922   756.549107   690.659099   
2023-12-09   777.647984   761.374647  1173.950743   823.378343   872.250125   
2023-12-16   882.112043   863.431859  1541.179554   927.821137  1039.835763   
2023-12-23  1022.594051  1003.760216  1792.010500  1070.670933  1152.312478   
2023-12-30  1194.045236  1178.908972  1809.615902  1246.685908  1176.025820   
2024-01-06  1380.262440  1374.229966  1591.156576  1438.943701  1104.679875   

                   3664  
date                     
2023-11-25   997.664038  
2023-12-02   970.555943  
2023-12-09   982.534526  
2023-12-16  1036.105404  
2023-12-23  1131.456727  
2023-12-30  1266.702337  
2024-01-06  1433.673228  

[7 rows x 100 columns]

## Attempt of a forecast from the previous data and selected parameters

All cells above must be run before. This is a draft.

In [209]:
_rng = np.random.default_rng(1)

forecast_params = fs_initial_params.copy()
forecast_params.update(dict(
    t_init=report_date + pd.Timedelta("1d"),  # First day to run the model for
    roi_len=28,  # Number of days to forecast
))

# Dummy series just to call `calculate_loglikelihood` within the forecasting period
_dummy_truth_sr = pd.Series(
    1.,
    index=pd.date_range(report_date + pd.Timedelta("1w"), report_date + pd.Timedelta(forecast_params["roi_len"], "d"), freq="W-SAT")
)


_sample_hosp_list = list()
_sample_rt_list = list()
_keys = list()
for i_sample, _row in _mcmc_rows.iterrows():
    # NOTE: This way will probably have a redundant day in the model. Need to fix that in a final version.

    # --- Get past data from the fitting phase
    # _past_mu_sr = _mu_runs_df[i_sample]
    _past_rt_sr = _rt_runs_df[i_sample]
    _past_ct_sr = _ct_runs_df[i_sample]

    # --- Define parameters for this run
    forecast_params.update(_row)
    forecast_params["r_init"] = _past_rt_sr.iloc[-1]  # Start from the last R(t)

    # --- Run infection model, then do the infection-to-hospitalization sampling
    _ct_sr, _rt_sr = run_model_once(ct_burnin_sr=_past_ct_sr, **forecast_params)
    _mu_sr, _total_ll = calculate_loglikelihood(_ct_sr, _rt_sr, _dummy_truth_sr, **forecast_params)
    #  ^ I only call this function to get the _mu_sr. The likelihood is really not needed here. 

    # Negative Binomial sampling from the expected values
    _nb_n = 1. / forecast_params["likelihood_overdisp"]
    _nb_p = _nb_n / (_nb_n + _mu_sr)
    _sample_hosp_sr = pd.Series(
        scipy.stats.nbinom.rvs(n=_nb_n, p=_nb_p, random_state=_rng), index=_mu_sr.index
    )
    
    _sample_hosp_list.append(_sample_hosp_sr)
    _sample_rt_list.append(_rt_sr)
    _keys.append(i_sample)
    

sample_fore_hosp_df = pd.concat(_sample_hosp_list, axis=1, keys=_keys)
sample_fore_rt_df = pd.concat(_sample_rt_list, axis=1, keys=_keys)

sample_fore_hosp_df
    
    

2835  976   3343  4519  4185  2694  2238  536   60    2680  ...  \
2024-01-13  1708  1526  1498  1675  1852  1858  1759  1532  1704  1884  ...   
2024-01-20  1673  1679  1490  2005  2158  2301  1891  1754  1047  1937  ...   
2024-01-27  1868  1562  1324  2191  2769  2434  1573  1702   783  2291  ...   
2024-02-03  1645  1382  1355  2025  2576  2036  1766  1681   895  2129  ...   

            453   2741  2755  2184  2253  3026  275   2463  263   3664  
2024-01-13  1085  1878  1980  1522  1493  1488   941  1560   426  1231  
2024-01-20   899  2474  2350  2023  1562  1698  1258  1678   650  2006  
2024-01-27   808  2331  2228  1859  2261  1800   758  1711   497  1746  
2024-02-03  1014  2187  2361  2203  1747  1743   441  1941   481  2137  

[4 rows x 100 columns]

In [217]:
fig = plotly.subplots.make_subplots(rows=2, cols=1, row_heights=[100, 100], shared_xaxes=True)    

# # Add trajectory plots
# for (i_mu, i_rt) in zip(_mu_runs_df, _rt_runs_df):
#     _mu_sr = _mu_runs_df[i_mu]
#     _rt_sr = _rt_runs_df[i_rt]
#     fig.add_scatter(
#         x=_mu_sr.index, y=_mu_sr.values, row=1, col=1, name=f"Model {i} hosp.",
#         mode="lines", line_color="gray", opacity=0.1,
#         legendgroup=f"Sample {i}",
#     )
#     fig.add_scatter(
#         x=_rt_sr.index, y=_rt_sr.values, row=2, col=1, name=f"Model {i} R(t)",
#         legendgroup=f"Sample {i}", line_color="darkred", opacity=0.05
#     )

# Add quantile plots - Hospitalizations
_sr = _mu_runs_df.median(axis=1)
fig.add_scatter(x=_sr.index, y=_sr.values, row=1, col=1, name="Model Median Fit Hosp", mode="lines")
plotly_fill_between(x=_sr.index, 
                    y1=_mu_runs_df.quantile(0.025, axis=1).values,
                    y2=_mu_runs_df.quantile(0.975, axis=1).values,
                    fig=fig, row=1, col=1, name="Model 95% IQR Hosp", line_width=0., fillcolor="olivedrab",)


# FORECAST - Quantiles
_sr = sample_fore_hosp_df.median(axis=1)
fig.add_scatter(x=_sr.index, y=_sr.values, row=1, col=1, name="Model Median Forecast Hosp", mode="lines", line_color="darkblue")
plotly_fill_between(x=_sr.index, 
                    y1=sample_fore_hosp_df.quantile(0.25, axis=1).values,
                    y2=sample_fore_hosp_df.quantile(0.75, axis=1).values,
                    fig=fig, row=1, col=1, name="Model 50% IQR Forecast Hosp", line_width=0., fillcolor="darkblue",)
plotly_fill_between(x=_sr.index, 
                    y1=sample_fore_hosp_df.quantile(0.025, axis=1).values,
                    y2=sample_fore_hosp_df.quantile(0.975, axis=1).values,
                    fig=fig, row=1, col=1, name="Model 95% IQR Forecast Hosp", line_width=0., fillcolor="darkblue",)

# Add quantile plots - R(t)
_sr = _rt_runs_df.median(axis=1)
fig.add_scatter(x=_sr.index, y=_sr.values, row=2, col=1, name="Model Median Fit R(t)", mode="lines")
plotly_fill_between(x=_sr.index, 
                    y1=_rt_runs_df.quantile(0.025, axis=1).values,
                    y2=_rt_runs_df.quantile(0.975, axis=1).values,
                    fig=fig, row=2, col=1, name="Model 95% IQR R(t)", line_width=0., fillcolor="darkred",)

# Forecasted R(t)
_sr = sample_fore_rt_df.median(axis=1)
fig.add_scatter(x=_sr.index, y=_sr.values, row=2, col=1, name="Forecast R(t)", mode="lines", line_color="crimson")
plotly_fill_between(x=_sr.index, 
                    y1=sample_fore_rt_df.quantile(0.025, axis=1).values,
                    y2=sample_fore_rt_df.quantile(0.975, axis=1).values,
                    fig=fig, row=2, col=1, name="R(t) 95% IQR Forecast ", line_width=0., fillcolor="crimson",)


fig.add_hline(y=1, row=2, col=1, line_color="black")

    
# Plot truth series and stuff
# ============
_truth_empir_plot_sr = (
    truth_big_df.xs((location_name, report_date + pd.Timedelta("4w")), level=["location_name", "report_date"])["value"]
    .sort_index()
    .loc[_fit_window_start - pd.Timedelta("10w"):]
)
fig.add_scatter(
    x=_truth_empir_plot_sr.index, 
    y=_truth_empir_plot_sr.values, 
    row=1, col=1, name="4w later Empirical data")

_empir_plot_sr = _raw_incid_sr.loc[_fit_window_start - pd.Timedelta("10w"):report_date + pd.Timedelta("4w")]
fig.add_scatter(
    x=_empir_plot_sr.index, 
    y=_empir_plot_sr.values, 
    row=1, col=1, name="Empirical data")

fig.update_layout(height=800)

fig.show()

# Drafts

In [50]:
# Quick gamma calculator
gamma_mean = 5
gamma_std = 1

gamma_shape = gamma_mean**2 / gamma_std**2
gamma_rate = gamma_mean / gamma_std**2

gamma_shape, gamma_rate

(25.0, 5.0)

22610726

In [221]:
display(truth_big_df.xs((location_name, report_date), level=["location_name", "report_date"]).sort_index()["value"])

date
2022-02-12      68
2022-02-19      66
2022-02-26      73
2022-03-05      72
2022-03-12      68
              ... 
2023-12-09     763
2023-12-16     841
2023-12-23     986
2023-12-30    1328
2024-01-06    1396
Name: value, Length: 100, dtype: int64